In [ ]:
import numpy as np
import pandas as pd
from typing import List, Optional, Tuple
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import BayesianGaussianMixture

def fit_packing_cracking_mixture(
    cd_enacted: pd.DataFrame,
    feature_cols: Optional[List[str]] = None,
) -> Tuple[BayesianGaussianMixture, pd.DataFrame, List[str]]:
    df = cd_enacted.copy()

    if feature_cols is None:
        feature_cols = [
            "minority_share",
            "minority_dispersion_var",
            "minority_dispersion_gini",
            "minority_top10_mean",
        ]
        feature_cols += [c for c in df.columns if c.startswith("compactness_")]

    X = (df[feature_cols]
         .replace([np.inf, -np.inf], np.nan)
         .fillna(df[feature_cols].median(numeric_only=True))
         .values)

    Xs = StandardScaler().fit_transform(X)

    bgm = BayesianGaussianMixture(
        n_components=3,
        covariance_type="full",
        weight_concentration_prior_type="dirichlet_process",
        weight_concentration_prior=0.5,
        random_state=42,
    )
    bgm.fit(Xs)

    probs = bgm.predict_proba(Xs)

    out = df[["cd"]].copy()
    for k in range(probs.shape[1]):
        out[f"mix_comp_{k}_prob"] = probs[:, k]

    return bgm, out, feature_cols
